In [ ]:
# ========================================
# Pinecone 인덱스 내용 확인 및 분석
# 노트북에서 실행
# ========================================

import os
import logging
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

# 환경변수 로드
load_dotenv()

# ========================================
# 공통 클라이언트 초기화
# ========================================

def init_clients(index_name: str):
    """Pinecone 클라이언트와 OpenAI 임베딩 모델을 초기화"""
    
    # Pinecone 클라이언트
    api_key = os.getenv("PINECONE_API_KEY")
    if not api_key:
        raise ValueError("PINECONE_API_KEY가 설정되지 않았습니다.")
    
    pc = Pinecone(api_key=api_key)
    index = pc.Index(index_name)
    
    # OpenAI 임베딩 모델
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # Pinecone Index 모델과 같은 모델
    
    # VectorStore 객체
    vectorstore = PineconeVectorStore(index=index, embedding=embeddings)
    
    return pc, index, embeddings, vectorstore


# ========================================
# 주요 분석 함수들
# ========================================

def inspect_pinecone_index(pc, index, vectorstore, index_name: str):
    """Pinecone 인덱스의 내용을 상세하게 확인"""
    
    print(f"🔍 Pinecone 인덱스 '{index_name}' 내용 분석")
    print("=" * 60)
    
    try:
        print(f"✅ 인덱스 '{index_name}' 연결 성공")
        
        # 2. 인덱스 기본 통계
        print("\n2️⃣ 인덱스 기본 통계")
        stats = index.describe_index_stats()
        
        total_vectors = stats.get('total_vector_count', 0)
        dimension = stats.get('dimension', 'N/A')
        index_fullness = stats.get('index_fullness', 0)
        
        print(f"📊 기본 정보:")
        print(f"   - 총 벡터 수: {total_vectors:,}")
        print(f"   - 벡터 차원: {dimension}")
        print(f"   - 인덱스 사용률: {index_fullness:.2%}")
        
        # 3. 네임스페이스 정보
        print("\n3️⃣ 네임스페이스 정보")
        namespaces = stats.get('namespaces', {})
        
        if namespaces:
            for ns_name, ns_stats in namespaces.items():
                ns_count = ns_stats.get('vector_count', 0)
                print(f"   📁 네임스페이스 '{ns_name}': {ns_count:,}개 벡터")
        else:
            print("   📁 기본 네임스페이스만 사용 중")
        
        if total_vectors == 0:
            print("\n⚠️  인덱스가 비어있습니다.")
            return
        
        # 4. 샘플 벡터 조회 (모든 키워드 결과 출력)
        print(f"\n4️⃣ 샘플 벡터 조회")
        sample_queries = ["복지", "휴가", "정책", "직원", "회사"]
        
        for i, query in enumerate(sample_queries, 1):
            try:
                results = vectorstore.similarity_search(query, k=2)
                if results:
                    doc = results[0]
                    source = doc.metadata.get('source', 'Unknown')
                    content_preview = doc.page_content[:150].replace('\n', ' ').strip()
                    
                    print(f"\n   🔍 샘플 {i}: 검색어 '{query}'")
                    print(f"      📄 문서 출처: {source}")
                    print(f"      📝 내용 미리보기: {content_preview}...")
                    
                    # 메타데이터 표시
                    if doc.metadata:
                        print(f"      🏷️  메타데이터: {dict(doc.metadata)}")
                else:
                    print(f"\n   ⚠️  샘플 {i}: 검색어 '{query}' - 결과 없음")
                        
            except Exception as e:
                print(f"\n   ❌ 샘플 {i}: 검색어 '{query}' - 조회 실패 ({e})")
        
        # 5. 검색 성능 테스트
        print(f"\n5️⃣ 검색 성능 테스트")
        _run_performance_test(vectorstore)
        
        # 6. 문서 출처 분석
        print(f"\n6️⃣ 문서 출처 분석")
        _analyze_document_sources(vectorstore)
            
        print(f"\n" + "=" * 60)
        print("🎉 인덱스 분석 완료!")
        
    except Exception as e:
        print(f"❌ 인덱스 분석 실패: {e}")


def get_vector_samples(vectorstore, index_name: str, limit: int = 5):
    """벡터 샘플을 더 자세히 조회"""
    
    print(f"\n🔬 벡터 샘플 상세 분석 (최대 {limit}개)")
    print("=" * 50)
    
    try:
        # 광범위한 검색으로 다양한 문서 수집
        diverse_results = vectorstore.similarity_search("회사 정책 규정 혜택", k=limit)
        
        for i, doc in enumerate(diverse_results, 1):
            print(f"\n📄 문서 {i}:")
            print(f"   출처: {doc.metadata.get('source', 'Unknown')}")
            print(f"   길이: {len(doc.page_content)}자")
            
            # 메타데이터 상세 표시
            if doc.metadata:
                print(f"   메타데이터:")
                for key, value in doc.metadata.items():
                    print(f"      {key}: {value}")
            
            # 내용 표시 (처음 300자)
            content = doc.page_content[:300].replace('\n', '\n      ')
            print(f"   내용:")
            print(f"      {content}...")
            print("   " + "-" * 40)
            
    except Exception as e:
        print(f"❌ 벡터 샘플 조회 실패: {e}")


def check_index_health(index, vectorstore, index_name: str, min_content_length: int = 50):
    """인덱스 건강 상태 체크 (품질 기준 조절 가능)"""
    
    print(f"\n🏥 인덱스 '{index_name}' 건강 상태 체크")
    print(f"   (최소 내용 길이 기준: {min_content_length}자)")
    print("=" * 50)
    
    try:
        # 1. 기본 통계
        stats = index.describe_index_stats()
        total_vectors = stats.get('total_vector_count', 0)
        
        print(f"📊 기본 지표:")
        print(f"   - 벡터 수: {total_vectors}")
        print(f"   - 상태: {'✅ 정상' if total_vectors > 0 else '⚠️ 비어있음'}")
        
        if total_vectors == 0:
            return
        
        # 2. 검색 응답성 테스트
        import time
        test_query = "테스트"
        
        start = time.time()
        results = vectorstore.similarity_search(test_query, k=1)
        response_time = time.time() - start
        
        print(f"   - 응답 시간: {response_time:.3f}초 ({'✅ 빠름' if response_time < 1 else '⚠️ 느림'})")
        print(f"   - 검색 기능: {'✅ 정상' if results else '❌ 실패'}")
        
        # 3. 데이터 품질 체크 (조절 가능한 기준)
        quality_issues = 0
        sample_results = vectorstore.similarity_search("정책", k=5)
        
        for doc in sample_results:
            if len(doc.page_content) < min_content_length:
                quality_issues += 1
        
        quality_score = (len(sample_results) - quality_issues) / len(sample_results) * 100 if sample_results else 0
        print(f"   - 데이터 품질: {quality_score:.1f}% ({'✅ 우수' if quality_score > 80 else '⚠️ 개선필요' if quality_score > 60 else '❌ 문제'})")
        
        # 품질 상세 정보
        if quality_issues > 0:
            print(f"     * {min_content_length}자 미만 문서: {quality_issues}개")
        
    except Exception as e:
        print(f"❌ 건강 상태 체크 실패: {e}")


# ========================================
# 헬퍼 함수들
# ========================================

def _run_performance_test(vectorstore):
    """검색 성능 테스트 실행"""
    
    try:
        import time
        
        test_queries = [
            "휴가는 어떻게 신청하나요?",
            "복지 혜택에는 무엇이 있나요?",
            "회사 정책을 알고 싶어요"
        ]
        
        total_time = 0
        successful_searches = 0
        
        for query in test_queries:
            try:
                start_time = time.time()
                results = vectorstore.similarity_search(query, k=5)
                search_time = time.time() - start_time
                
                if results:
                    successful_searches += 1
                    total_time += search_time
                    print(f"   ✅ '{query}': {len(results)}개 결과 ({search_time:.3f}초)")
                else:
                    print(f"   ⚠️  '{query}': 결과 없음")
                    
            except Exception as e:
                print(f"   ❌ '{query}': 검색 실패 ({e})")
        
        if successful_searches > 0:
            avg_time = total_time / successful_searches
            print(f"\n   📊 평균 검색 시간: {avg_time:.3f}초")
            print(f"   📊 성공률: {successful_searches}/{len(test_queries)} ({successful_searches/len(test_queries)*100:.1f}%)")
            
    except Exception as e:
        print(f"   ❌ 성능 테스트 실패: {e}")


def _analyze_document_sources(vectorstore):
    """문서 출처 분석"""
    
    try:
        # 다양한 검색어로 모든 출처 파악
        all_sources = set()
        sample_docs_by_source = {}
        
        broad_queries = ["정책", "규정", "혜택", "직원", "회사", "업무"]
        
        for query in broad_queries:
            try:
                results = vectorstore.similarity_search(query, k=10)
                for doc in results:
                    source = doc.metadata.get('source', 'Unknown')
                    all_sources.add(source)
                    
                    # 출처별 샘플 문서 저장
                    if source not in sample_docs_by_source:
                        sample_docs_by_source[source] = doc.page_content[:100]
                        
            except Exception:
                continue
        
        print(f"   📁 발견된 문서 출처: {len(all_sources)}개")
        for source in sorted(all_sources):
            sample = sample_docs_by_source.get(source, "샘플 없음")
            print(f"      📄 {source}: {sample.replace(chr(10), ' ')}...")
            
    except Exception as e:
        print(f"   ❌ 출처 분석 실패: {e}")


def quick_check(index_name: str = "gaida-hr-rules"):
    """빠른 상태 확인 (독립적으로 동작)"""
    
    try:
        pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
        index = pc.Index(index_name)
        stats = index.describe_index_stats()
        
        total_vectors = stats.get('total_vector_count', 0)
        
        print(f"🚀 '{index_name}' 빠른 확인")
        print(f"   벡터 수: {total_vectors:,}")
        print(f"   상태: {'✅ 사용가능' if total_vectors > 0 else '⚠️ 비어있음'}")
        
        return total_vectors > 0
        
    except Exception as e:
        print(f"❌ 빠른 확인 실패: {e}")
        return False


# ========================================
# 메인 실행 함수들
# ========================================

def run_full_inspection(index_name: str = "gaida-hr-rules", min_content_length: int = 50):
    """전체 인덱스 검사 실행 (개선된 버전)"""
    
    try:
        # 한 번만 클라이언트 초기화
        print("1️⃣ 클라이언트 초기화 중...")
        pc, index, embeddings, vectorstore = init_clients(index_name)
        print("✅ 클라이언트 초기화 완료")
        
        # 기본 인덱스 분석
        inspect_pinecone_index(pc, index, vectorstore, index_name)
        
        # 벡터 샘플 상세 분석
        get_vector_samples(vectorstore, index_name, limit=3)
        
        # 건강 상태 체크 (조절 가능한 품질 기준)
        check_index_health(index, vectorstore, index_name, min_content_length)
        
    except Exception as e:
        print(f"❌ 전체 검사 실패: {e}")


def run_basic_inspection(index_name: str = "gaida-hr-rules"):
    """기본 인덱스 분석만 실행"""
    
    try:
        pc, index, embeddings, vectorstore = init_clients(index_name)
        inspect_pinecone_index(pc, index, vectorstore, index_name)
        
    except Exception as e:
        print(f"❌ 기본 검사 실패: {e}")


def run_custom_inspection(index_name: str = "gaida-hr-rules", 
                         sample_limit: int = 5, 
                         min_content_length: int = 30):
    """사용자 정의 설정으로 검사 실행"""
    
    try:
        pc, index, embeddings, vectorstore = init_clients(index_name)
        
        print(f"🔧 사용자 정의 설정:")
        print(f"   - 인덱스: {index_name}")
        print(f"   - 샘플 제한: {sample_limit}개")
        print(f"   - 최소 내용 길이: {min_content_length}자")
        
        # 맞춤형 분석 실행
        inspect_pinecone_index(pc, index, vectorstore, index_name)
        get_vector_samples(vectorstore, index_name, limit=sample_limit)
        check_index_health(index, vectorstore, index_name, min_content_length)
        
    except Exception as e:
        print(f"❌ 맞춤형 검사 실패: {e}")

c:\Users\AY\Desktop\PROJECTS\HR-chatbot-development-project\venv\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [2]:
# ========================================
# 노트북에서 실행
# ========================================

if __name__ == "__main__":
    # 전체 분석 실행 (개선된 버전)
    run_full_inspection()
    
    # 또는 개별/맞춤형 실행:
    # run_basic_inspection()                              # 기본 분석만
    # run_custom_inspection(sample_limit=10, min_content_length=30)  # 맞춤형 설정
    # quick_check()                                       # 빠른 확인

1️⃣ 클라이언트 초기화 중...
✅ 클라이언트 초기화 완료
🔍 Pinecone 인덱스 'gaida-hr-rules' 내용 분석
✅ 인덱스 'gaida-hr-rules' 연결 성공

2️⃣ 인덱스 기본 통계
📊 기본 정보:
   - 총 벡터 수: 15
   - 벡터 차원: 1536
   - 인덱스 사용률: 0.00%

3️⃣ 네임스페이스 정보
   📁 네임스페이스 '': 15개 벡터

4️⃣ 샘플 벡터 조회

   🔍 샘플 1: 검색어 '복지'
      📄 문서 출처: 04_복지정책_v1.0.md
      📝 내용 미리보기: ## 2. 복지포인트 및 지원금   ### 2.1 복지포인트 - 연간 300만원의 복지포인트가 지급됩니다. - 당해년도 입사자(1년 미만 근속자)는 근무 개월 수에 비례하여 월할 계산하여 당해분 복지포인트가 지급됩니다. (지급일: 수습기간 종료일) - AK복지몰(htt...
      🏷️  메타데이터: {'doc_title': '가이다 플레이 스튜디오(GPS) 직원 복지제도 종합 안내서', 'main_category': '2. 복지포인트 및 지원금', 'source': '04_복지정책_v1.0.md', 'sub_category': '2.1 복지포인트'}

   🔍 샘플 2: 검색어 '휴가'
      📄 문서 출처: 04_복지정책_v1.0.md
      📝 내용 미리보기: ## 1. 휴가 및 휴직 제도   ### 1.1 연차휴가 - 연차휴가는 기본 15일입니다. - 기존 근속자를 대상으로 매년 1월 1일 부여됩니다. (단, 전년도 출근율이 80% 이상이어야 발생) - 발생한 연차는 해당년도 12월 31일까지 사용해야 하며, 미사용 연차는...
      🏷️  메타데이터: {'doc_title': '가이다 플레이 스튜디오(GPS) 직원 복지제도 종합 안내서', 'main_category': '1. 휴가 및 휴직 제도', 'source': '04_복지정책_v1.0.md', 'sub_category': '1.1 연차휴가'}

   🔍 